In [5]:
from sklearn.feature_extraction import DictVectorizer
import pymysql
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random
from langdetect import detect
import csv
import MySQLdb


In [27]:
# load data form database
conn = MySQLdb.connect(host='127.0.0.1', user='root', passwd="root", db='paraphrases')
cur = conn.cursor(MySQLdb.cursors.DictCursor)
cur.execute("SELECT * FROM plagiat ORDER BY author")

originals = []
plagiats = []

groups = {}

i=0
for r in cur:
    
    originals.append(r["source_text"])
    plagiats.append(r["plagiat_text"])
    group = r["author"]
    if not group in groups:
        groups[group] = []
    groups[group].append(i)
    i+=1
  

cur.close()
conn.close()

In [14]:
print(plagiats[0])

1. EINLEITUNG Staphylococcus aureus ist mit seinen Methicillin-resistenten Stämmen ein häufiger Verursacher von Infektionen in Krankenhäusern und anschwellend auch in der Bevölkerung. Bereits 1961 wurde sein Auftreten in Großbritannien entdeckt, lediglich zwei Jahre nach der Introduktion des semisynthetischen Penicillin Methicillin, welches zur Behandlung von Penicillin-resistentem S. aureus genutzt wurde [1]. Zum ersten großen Auftakt von MRSA kam es 1967 in einem Krankenhaus in Boston [2,3]. Die MRSA-Prävalenz ist seither weltweit anhaltend gestiegen. Allerdings ist die Verteilung sowohl national als auch regional sehr heterogen. Einzig in den USA starben im Jahr 2006 circa 19.000 Menschen an MRSA-Infektionen. Im Jahr 2005 reichte die Zahl der stationär behandelten MRSA-Infektionen bis zu 278.000. Im Vergleich zu 1999 in dem 127.000 Fälle beschrieben wurden, zeigte sich eine Verdopplung [4,5]. Schätzungsweise sterben in Deutschland zwischen 700 und 1.500 Personen an einer MRSA-Infekt

In [23]:
# language detection
conn = MySQLdb.connect(host='127.0.0.1', user='root', passwd="root", db='paraphrases')
cur = conn.cursor(MySQLdb.cursors.DictCursor)
cur.execute("SELECT * FROM plagiat ORDER BY author")

for r in cur:
    
    try:
        url = r["url"]
        original = r["source_text"]
        plagiat = r["plagiat_text"]

        lang_original = detect(original)
        lang_plagiat = detect(plagiat)

        query = "UPDATE plagiat SET lang_source='" + lang_original + "', lang_plagiat='" + lang_plagiat;
        query += "' WHERE url='" + url + "';";
        cur2 = conn.cursor()
        cur2.execute(query)
        cur2.close()
        
    except Exception:
        print("Exception: " + url)
conn.commit();
cur.close()
conn.close()

Exception: http://de.vroniplag.wikia.com/wiki/Aaf/Fragment_028_01
Exception: http://de.vroniplag.wikia.com/wiki/Ab/Fragment_150_02
Exception: http://de.vroniplag.wikia.com/wiki/Ast/Fragment_141_105
Exception: http://de.vroniplag.wikia.com/wiki/Ast/Fragment_145_103
Exception: http://de.vroniplag.wikia.com/wiki/Bes/Fragment_034_02
Exception: http://de.vroniplag.wikia.com/wiki/Bes/Fragment_035_00
Exception: http://de.vroniplag.wikia.com/wiki/Bes/Fragment_036_00
Exception: http://de.vroniplag.wikia.com/wiki/Bes/Fragment_037_00
Exception: http://de.vroniplag.wikia.com/wiki/Bes/Fragment_038_00
Exception: http://de.vroniplag.wikia.com/wiki/Bes/Fragment_039_00
Exception: http://de.vroniplag.wikia.com/wiki/Bes/Fragment_040_00
Exception: http://de.vroniplag.wikia.com/wiki/Bes/Fragment_042_00
Exception: http://de.vroniplag.wikia.com/wiki/Bes/Fragment_045_01
Exception: http://de.vroniplag.wikia.com/wiki/Bes/Fragment_045_02
Exception: http://de.vroniplag.wikia.com/wiki/Bes/Fragment_046_01
Exception

In [28]:
# read stopwords
stopwords = set()
with open("stopwords_de.txt") as f:
    content = f.readlines()
    for line in content:
        stopwords.update(line.replace("\n", ""))


# do tf idf
all_docs = originals + plagiats
v = TfidfVectorizer(analyzer="word", stop_words = stopwords, ngram_range=(1,1), max_df=1000, min_df=3)
tfidf = v.fit_transform(all_docs)


In [29]:
# evaluation - target + 19 random docs from the same author
test_length = 50
correct = 0

for key in groups.keys():
    
    eval_size = len(groups[key])
    test_length = min(20, len(groups[key]))
    correctInGroup = 0

    for i in range(0, eval_size ):

        # generate a test set of the original and 19 random text documents

        index = groups[key][i]
        originalDoc = tfidf.getrow(index)

        test_docs = []    
        test_docs.append(tfidf.getrow(len(originals)+index))
        usedIndizes = {}
        usedIndizes[index] = True

        while len(test_docs) < test_length:

            j = random.randrange(0, eval_size)
            if not j in usedIndizes:
                index = groups[key][j]
                usedIndizes[index] = True
                test_docs.append(tfidf.getrow(index))

        # compare test set
        maxSim = 0
        mostSimDoc = 0
        for j in range(0, len(test_docs)):
            vecS = test_docs[j]
            sim = cosine_similarity(originalDoc, vecS)
            sim = sim[0][0]

            if(maxSim < sim):
                maxSim=sim
                mostSimDoc = j

        if mostSimDoc == 0:
            correctInGroup += 1
    
    correct += correctInGroup
    accuracy = correctInGroup / float(eval_size)
    print(key + "\t" + str(eval_size) + "\t" + str(accuracy))
    
accuracy = correct/float(len(originals))
print("Total: \t" + str(accuracy))


Nm3	12	0.3333333333333333
Hk	16	0.3125
Ntx	19	0.47368421052631576
Nm6	13	0.3076923076923077
Sse	173	0.884393063583815
Aos	89	0.7865168539325843
Bza	23	0.5217391304347826
Ahg	44	0.7727272727272727
Tz	20	0.45
Tn	90	0.7888888888888889
Tt	52	0.7115384615384616
Lr	26	0.34615384615384615
Qkn	10	0.3
Ao	54	0.5740740740740741
Is	135	0.8740740740740741
Or	21	0.38095238095238093
Tf	15	0.6
Alm	33	0.48484848484848486
Ik	26	0.23076923076923078
Gdp	35	0.5714285714285714
Ks	93	0.7849462365591398
Dm	60	0.7333333333333333
Jm	127	0.8503937007874016
Kml	29	0.7241379310344828
Mhe	126	0.8174603174603174
Tr	297	0.9326599326599326
Ahh	4	0.5
Tmu	76	0.8421052631578947
Ak	24	0.4166666666666667
Sh	79	0.7215189873417721
Nkl	22	0.4090909090909091
Al	31	0.6451612903225806
Qf	25	0.52
Db	302	0.9172185430463576
Cad	37	0.6756756756756757
Cep	22	0.5
Xg	144	0.8611111111111112
Kal	28	0.6071428571428571
Kk	21	0.47619047619047616
Cak	17	0.5882352941176471
Rsi	20	0.5
Nm	228	0.9166666666666666
Gp	115	0.8695652173913043
Nm4	5	0

In [30]:
# evaluation - target + 19 random docs

test_length = 20
correct = 0
eval_size = len(originals)
for i in range(0, eval_size ):
    
    if i % 500 == 0:
        print("Processed " + str(i) + " docs")
        
    # generate a test set of the original and 19 random text documents
    
    originalDoc = tfidf.getrow(i)
    
    test_docs = []    
    test_docs.append(tfidf.getrow(len(originals)+i))
    usedIndizes = {}
    
    while len(test_docs) < test_length:
        
        j = random.randrange(len(originals), 2*len(originals))
        if not j in usedIndizes:
            usedIndizes[j] = True
            test_docs.append(tfidf.getrow(j))
            
    # compare test set
    maxSim = 0
    mostSimDoc = 0
    for j in range(0, len(test_docs)):
        vecS = test_docs[j]
        sim = cosine_similarity(originalDoc, vecS)
        sim = sim[0][0]
        
        if(maxSim < sim):
            maxSim=sim
            mostSimDoc = j

    if mostSimDoc == 0:
        correct += 1
       
    

print(correct/float(eval_size)) 

Processed 0 docs
Processed 500 docs
Processed 1000 docs
Processed 1500 docs
Processed 2000 docs
Processed 2500 docs
Processed 3000 docs
Processed 3500 docs
Processed 4000 docs
Processed 4500 docs
Processed 5000 docs
Processed 5500 docs
Processed 6000 docs
Processed 6500 docs
Processed 7000 docs
Processed 7500 docs
Processed 8000 docs
Processed 8500 docs
Processed 9000 docs
Processed 9500 docs
Processed 10000 docs
Processed 10500 docs
Processed 11000 docs
Processed 11500 docs
Processed 12000 docs
Processed 12500 docs
Processed 13000 docs
Processed 13500 docs
Processed 14000 docs
Processed 14500 docs
Processed 15000 docs
Processed 15500 docs
0.9970823290625397


In [25]:
# convert to csv

conn = MySQLdb.connect(host='127.0.0.1', user='root', passwd="root", db='paraphrases')
cur = conn.cursor(MySQLdb.cursors.DictCursor)
cur.execute("SELECT * FROM plagiat ORDER BY author")

keys = { 
    "url",
    "fragment_identifier",
    "author",
    "source_text",
    "full_html",
    "category",
    "lang_source",
    "lang_plagiat",
    "plagiat_text",
    "peer_reviewed"
}

counter = 0
with open('vroniplag-corpus.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter='\t',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    spamwriter.writerow(keys)
    
    for r in cur:
        if(r["peer_reviewed"] == b'\x00'):
            r["peer_reviewed"] = 1;
        else:
            r["peer_reviewed"] = 0;
        row = []
        for key in keys:
            row.append(r[key])
        spamwriter.writerow(row)
        counter += 1
  
print(counter)

cur.close()
conn.close()

15758
